# Machine Learning Course 2025 HW2
The code scripts are from [aideml](https://github.com/WecoAI/aideml) project on github with some modifications.

AIDE: AI-Driven Exploration in the Space of Code

https://arxiv.org/pdf/2502.13138


<font color='red' size=6>Make a copy before running or editing the code.</font>

## Prerequisites

In [ ]:
# check GPU
!nvidia-smi

Sun Apr 13 01:19:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   47C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# install packages
!pip install dataclasses_json==0.6.4 shutup==0.2.0
!pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 300.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 208.3 MB/s eta 0:00:00


In [ ]:
# Download dataset
!gdown --id 1Ah5uV6cu3Bnz6WfkUuxEZCLqj5k1lbpd
# Choose a workable link
# !gdown --id 1XtF9-hGw2tKe4WvUMW5YE6lj6p1QcWIc
# !gdown --id 1diswE_9XoT-uII23ucRppau1ErEQkY2y
# !gdown --id 1BAVMzLZqEgtG8rwog7ttC7xKPw5QTngn
# !gdown --id 1PAI4_3kRWwIPQMscMdGt9HLqZZy1vWSD
!unzip /content/ML2025Spring-hw2-public.zip
# 实验室不能连接外网，所以没有用该方法获取数据集

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ah5uV6cu3Bnz6WfkUuxEZCLqj5k1lbpd
To: /content/ML2025Spring-hw2-public.zip
100% 621k/621k [00:00<00:00, 131MB/s]
Archive:  /content/ML2025Spring-hw2-public.zip
   creating: ML2025Spring-hw2-public/
  inflating: ML2025Spring-hw2-public/sample_submission.csv  
  inflating: ML2025Spring-hw2-public/test.csv  
  inflating: ML2025Spring-hw2-public/train.csv  


In [ ]:
# ========================== TODO: try different LLM ==========================
# Hugging Face: https://huggingface.co/models?library=gguf
# OpenLLM Leaderboard: https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/?params=7%2C65&official=true
# remember to replace 'blob' with 'resolve' in the link you copy.
!wget https://huggingface.co/bartowski/Qwen2.5-14B-Instruct-1M-GGUF/resolve/main/Qwen2.5-14B-Instruct-1M-Q5_K_S.gguf
# 实验室不能连接外网，所以没有用该方法获取大模型

--2025-04-13 01:19:28--  https://huggingface.co/bartowski/Qwen2.5-14B-Instruct-1M-GGUF/resolve/main/Qwen2.5-14B-Instruct-1M-Q5_K_S.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.97, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/aa/f7/aaf7878389076edd8ae82aabbe3cba7c64452c86e6ad848152e8513d966d62fa/ed5083190ac29b3587f08837063bb9003250e69048ed0c7936401780edb6ed44?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Qwen2.5-14B-Instruct-1M-Q5_K_S.gguf%3B+filename%3D%22Qwen2.5-14B-Instruct-1M-Q5_K_S.gguf%22%3B&Expires=1744510768&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDUxMDc2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2FhL2Y3L2FhZjc4NzgzODkwNzZlZGQ4YWU4MmFhYmJlM2NiYTdjNjQ0NTJjODZlNmFkODQ4MTUyZTg1MTNkOTY2ZDYyZmEvZWQ1MDgzMTkwYWMyOWIzNTg3ZjA4ODM

In [ ]:
import os
import sys
import torch
import subprocess

# 检查 CUDA 是否可用
print("CUDA 是否可用:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA 版本:", torch.version.cuda)
    print("GPU 数量:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"GPU {i} 内存总量: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

# 检查系统信息
print("\n系统信息:")
print("Python 版本:", sys.version)
print("操作系统:", os.name)

# 尝试检查 llama.cpp 版本或配置
try:
    result = subprocess.run(["pip", "show", "llama-cpp-python"],
                            capture_output=True, text=True)
    print("\nllama-cpp-python 信息:")
    print(result.stdout)
except Exception as e:
    print("无法获取 llama-cpp-python 信息:", e)

# 检查 CPU 信息
try:
    if os.name == 'posix':  # Linux/Mac
        cpu_info = subprocess.run(["lscpu"], capture_output=True, text=True)
        print("\nCPU 信息:")
        print(cpu_info.stdout)
    elif os.name == 'nt':  # Windows
        cpu_info = subprocess.run(["wmic", "cpu", "get", "name"], capture_output=True, text=True)
        print("\nCPU 信息:")
        print(cpu_info.stdout)
except Exception as e:
    print("无法获取 CPU 信息:", e)


CUDA 是否可用: True
CUDA 版本: 12.4
GPU 数量: 1
GPU 0: NVIDIA L4
GPU 0 内存总量: 22.16 GB

系统信息:
Python 版本: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
操作系统: posix

llama-cpp-python 信息:
Name: llama_cpp_python
Version: 0.3.4
Summary: Python bindings for the llama.cpp library
Home-page: https://github.com/abetlen/llama-cpp-python
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


CPU 信息:
Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Address sizes:                        46 bits physical, 48 bits virtual
Byte Order:                           Little Endian
CPU(s):                               12
On-line CPU(s) list:                  0-11
Vendor ID:                            GenuineIntel
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
CPU family:                           6
Model:     

In [ ]:
from llama_cpp import Llama

# Load the model onto GPU
myModel = Llama(
    # ========================== TODO: try different LLM ==========================
    # Before changing LLM, restart the session!
    "/content/Qwen2.5-14B-Instruct-1M-Q5_K_S.gguf",
    verbose=False,
    n_gpu_layers=-1,  # 使用所有可用的 GPU 层
    n_ctx=10024,
    offload_kqv=True,  # 帮助更好地利用 GPU
    n_batch=512       # 增加批处理大小可能提高 GPU 利用率
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=8000,    # This argument is how many tokens the model can generate.
        temperature=0.2      # This argument is the randomness of the model. 0 means no randomness. We suggest setting the temperature value to 0 for reproducibility.
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (10048) < n_ctx_train (1010000) -- the full capacity of the model will not be utilized


In [ ]:
# test_response1 = generate_response(myModel,"你是一个AI大模型，请给我介绍一下你是谁")
# test_response2 = generate_response(myModel,"请自我介绍一下,告诉我你是谁")
# test_response3 = generate_response(myModel,"请自我介绍一下,告诉我你是谁")
# print(test_response1)
# print(test_response2)
# print(test_response3)

## Functions

### Utils

In [ ]:
# Define a function to save the best solution and other good solutions to files.
def save_run(cfg, journal): #cfg配置对象，journal是来自自定义的 Journal 类的实例
    # Retrieve and save the best found solution.
    best_node = journal.get_best_node(only_good=False)  # Get the best node.
    with open("best_solution.py", "w") as f:
        f.write(best_node.code)

    good_nodes = journal.get_good_nodes()  # Retrieve all good solution nodes.
    for i, node in enumerate(good_nodes):
        filename = f"good_solution_{i}.py"
        with open(filename, "w") as f:
            f.write(node.code)

### Interpreter (DO NOT MODIFY THIS CELL)

In [ ]:
!pip install rich
!pip install humanize

In [ ]:
"""
DO NOT MODIFY THIS CELL

Python interpreter for executing code snippets and capturing their output.
"""


import logging
import os
import queue
import signal
import sys
import time
import traceback
import zipfile
from pathlib import Path
from shutil import rmtree
import shutil
from multiprocessing import Process, Queue
from typing import Hashable, cast

import humanize
import rich
import shutup
from rich.logging import RichHandler
from rich.syntax import Syntax
from dataclasses import dataclass
from dataclasses_json import DataClassJsonMixin


@dataclass #Python3.7的装饰器，类似于JS的annotation，可以自动给类添加一些方法来使得这个类具有数据存储的功能
class ExecutionResult(DataClassJsonMixin):
    """
    Result of executing a code snippet in the interpreter.
    Contains the output, execution time, and exception information.
    """
    term_out: list[str] #模型的输出结果（包括成功的代码或者报错信息）
    exec_time: float
    exc_type: str | None
    exc_info: dict | None = None
    exc_stack: list[tuple] | None = None
#由于具备装饰器，那么可以直接通过 ER1.term_out 来得到 ER1这个实例的term_out属性值。（否则还要写get_attr的方法）类似地也能更改、比较等

def exception_summary(e, exec_file_name):
    """Generates a string that summarizes an exception and its stack trace"""
    tb_lines = traceback.format_exception(e)
    #traceback是来自python的标准类中的方法，这个format_exception方法可以把报错信息以list的类型存储起来
    tb_str = "".join(
        [
            line
            for line in tb_lines
            #把这个list遍历一遍，就组成了报错信息
        ]
    )
    exc_info = {}
    if hasattr(e, "args"): #e是来自error大类（例如ValueError,TypeError等各类Python的错误类）的某一种实例
        exc_info["args"] = [str(i) for i in e.args]  # 这种实例自带很多属性，例如args就是其中一个，它是一个包含了信息的【元组】，和上面的traceback.format_exception有点像，但他是元组
    for att in ["name", "msg", "obj"]:
        if hasattr(e, att):
            exc_info[att] = str(getattr(e, att))
    #上面这个for循环是针对自定义错误类型的捕捉，不是python自带错误类型，但是从项目代码来看，项目直接就是使用compile执行了LLM生成的代码，所以到底哪里来的自定义错误呢？
    #或许是llama？待查证。
    #对于python的错误类型，只有比较简略的元组数据返回，例如错误
    """
    Traceback (most recent call last):
  File c:\\Users\\Administrator\\理解 exception_summary.py", line 70, in test_standard_exceptions
    print(my_list[10])
          ~~~~~~~^^^^
    IndexError: list index out of range
    """
    #对于这个错误的，就有元组:{'args': ['list index out of range']}
    #如果想让LLM帮我们改错误，这种程度的错误肯定是不够的，所以我们引入了自定义的错误类型，也就是用"name", "msg", "obj"更加详细地说明了错误
    #详见参考代码：理解 exception_summary.py

    tb = traceback.extract_tb(e.__traceback__)  # Extract the traceback information.
    # Create a list of tuples for each frame in the traceback.
    exc_stack = [(t.filename, t.lineno, t.name, t.line) for t in tb]

    return tb_str, e.__class__.__name__, exc_info, exc_stack  #总之最后是返回了一个具备完整报错信息和报错信息各个分类属性的信息，需要继续看一下到底是怎么定义这个e的。

# Define a class that redirects write operations to a multiprocessing queue.
class RedirectQueue:
    def __init__(self, queue, timeout=5):
        self.queue = queue  # Store the provided queue.
        self.timeout = timeout  # Set the timeout for queue operations.

    def write(self, msg):
        try:
            self.queue.put(msg, timeout=self.timeout)  # Attempt to put the message into the queue.
        except queue.Full:
            print.warning("Queue write timed out")  # Warn if the queue is full and the write times out.

    def flush(self):
        pass  # No operation is needed for flushing in this context.
        #这是一个用于捕获多进程环境下错误信息的类，是结合 Python的 multiprocessing 模块使用的
        #简单来说，他是捕获 Queue 类（来自multiprocessing模块）的信息的，这个信息来自 Process 类（来自multiprocessing模块）
        #详见示例代码 理解Queue.py

# Define the Interpreter class that simulates a standalone Python REPL.
class Interpreter: #执行LLM生成的代码
    def __init__(
        self,
        timeout: int = 1200,  # 程序（子进程）的执行总时长，超过此时长的程序将直接被切断
        agent_file_name: str = "runfile.py",  # Default file name for writing the agent's code.
    ):
        """
        Simulates a standalone Python REPL with an execution time limit.

        Args:
            timeout (int, optional): Timeout for each code execution step. Defaults to 3600.
            agent_file_name (str, optional): The name for the agent's code file. Defaults to "runfile.py".
        """
        self.timeout = timeout  # Save the timeout value.
        self.agent_file_name = agent_file_name  # Save the agent file name.
        self.process: Process = None  # 来自 Process 类的实例process（来自multiprocessing模块）

    def child_proc_setup(self, result_outq: Queue) -> None:
        import shutup
        shutup.mute_warnings()  # Mute all warnings before further execution.
        sys.stdout = sys.stderr = RedirectQueue(result_outq)
        #函数的作用是把 sys.stdout 和 sys.stderr 直接传给RedirectQueue
        #这个连等相当于:
        #temp =  RedirectQueue(result_outq)
        #sys.stderr = temp
        #sys.stdout = temp
        #目的就是把程序的所有输出（包括错误）传给RedirectQueue，便于LLM进行下一步的分析
        #shutup.mute_warnings()则是不想警告信息进来干扰执行，
        #如果你用过LLM你就知道LLM为了力求全面，很多时候都会对警告信息尝试性修复，但是这种修复很多时候都是没用的。

    def _run_session(
        self, code_inq: Queue, result_outq: Queue, event_outq: Queue
    ) -> None:
        self.child_proc_setup(result_outq)  # Set up the child process for capturing output.
        #创建子进程，使LLM生成的代码独立于main之外，出现了错误的代码也不会中断main
        global_scope: dict = {}  # Create an empty dictionary to serve as the global scope.
        while True:  # Continuously wait for new code to execute.
            code = code_inq.get()  # Retrieve code from the code input queue.
            with open(self.agent_file_name, "w") as f:  # Open the agent file for writing.
                f.write(code)  # Write the received code into the file.
            #这里为什么要写进一个文件是为了报错的时候知道具体是哪一行报错了，否则悬空放置的话不知道如何改错
            event_outq.put(("state:ready",))  # Signal that the interpreter is ready to execute the code.
            try:
                #这里直接调用了python自带的执行(exec)和编译(compile)来运行程序，那么上面说的自定义错误从何而来？
                exec(compile(code, self.agent_file_name, "exec"), global_scope)
                #一旦上面的try失败（也就是程序出错了），那么就开始使用exception_summary捕捉这个程序的错误
            except BaseException as e:
                tb_str, e_cls_name, exc_info, exc_stack = exception_summary(
                    e,
                    self.agent_file_name,
                )
                result_outq.put(tb_str)  # Put the traceback string into the result queue.
                if e_cls_name == "KeyboardInterrupt":
                    e_cls_name = "TimeoutError"  # Convert a KeyboardInterrupt into a TimeoutError.
                event_outq.put(("state:finished", e_cls_name, exc_info, exc_stack))  # Signal that execution finished with an error.
            else:
                event_outq.put(("state:finished", None, None, None))  # Signal that execution finished successfully.

            os.remove(self.agent_file_name)  # Remove the agent file after execution.

            result_outq.put("<|EOF|>")  # Put an EOF marker to indicate the end of output.

    def create_process(self) -> None:
        # Create three queues for communication with the child process:
        # - code_inq: for sending code to execute.
        # - result_outq: for receiving output from the execution.
        # - event_outq: for receiving state events (like ready and finished).
        # trunk-ignore(mypy/var-annotated)
        self.code_inq, self.result_outq, self.event_outq = Queue(), Queue(), Queue()
        #多线程处理中，Queue用于存放信息
        self.process = Process(
            target=self._run_session,  # Set the target function for the child process.
            args=(self.code_inq, self.result_outq, self.event_outq),  # Provide the necessary queues as arguments.
        )
        self.process.start()  # Start the child process.
        #开始利用 process 实例来执行程序

    def cleanup_session(self):
        if self.process is None:  # If there is no process, nothing to clean up.
            return
        try:
            # Attempt to terminate the child process gracefully.
            self.process.terminate()  # Request the process to terminate.
            self.process.join(timeout=0.5)  # Wait for the process to finish with a 0.5-second timeout.

            if self.process.exitcode is None:  # If the process is still running,
                self.process.kill()  # Forcefully kill the process.
                self.process.join(timeout=0.5)  # Wait again for termination.

                if self.process.exitcode is None:  # If the process still hasn't terminated,
                    os.kill(self.process.pid, signal.SIGKILL)  # Send a SIGKILL signal.
        except Exception as e:
            print(f"Error during process cleanup: {e}")  # Print an error message if cleanup fails.
        finally:
            if self.process is not None:  # If the process exists,
                self.process.close()  # Close the process.
                self.process = None  # Reset the process attribute to None.
        # 清理队列，确保没有残留数据
        if hasattr(self, "code_inq"):
            self.code_inq.close()
        if hasattr(self, "result_outq"):
            self.result_outq.close()
        if hasattr(self, "event_outq"):
            self.event_outq.close()

    def run(self, code: str, reset_session=True) -> ExecutionResult:
        """
        Execute the provided Python command in a separate process and return its output.

        Parameters:
            code (str): Python code to execute.
            reset_session (bool, optional): Whether to reset the interpreter session before executing the code. Defaults to True.

        Returns:
            ExecutionResult: Object containing the output and metadata of the code execution.
        """

        if reset_session:
            if self.process is not None:
                # If a previous process exists, clean it up before starting a new one.
                self.cleanup_session()
            self.create_process()  # Create a new child process.
        else:
            # For the first execution, reset_session must be True.
            assert self.process is not None

        assert self.process.is_alive()  # Ensure that the child process is running.

        self.code_inq.put(code)  # Send the code to the child process via the queue.
        #进程是空的的时候才把LLM的code进行执行
        # Wait for the child process to signal that it is ready.
        try:
            state = self.event_outq.get(timeout=10)  # Wait up to 10 seconds for the "state:ready" event.
        except queue.Empty:
            msg = "REPL child process failed to start execution"
            print.critical(msg)  # Log a critical error if the process does not start.
            while not self.result_outq.empty():
                continue  # Drain the result queue.
            raise RuntimeError(msg) from None
        assert state[0] == "state:ready", state  # Verify that the received state is "state:ready".
        start_time = time.time()  # Record the start time of execution.

        child_in_overtime = False  # Flag to indicate if the child process has exceeded the timeout.

        while True:
            try:
                # Try to get the finished state from the child process.
                state = self.event_outq.get(timeout=1)  # Wait for the "state:finished" event.
                assert state[0] == "state:finished", state  # Ensure the state is "state:finished".
                exec_time = time.time() - start_time  # Calculate the total execution time.
                break  # Exit the loop if execution is finished.
            except queue.Empty:
                # If no event is received, check whether the process is still alive.
                if not child_in_overtime and not self.process.is_alive():
                    msg = "REPL child process died unexpectedly"
                    raise RuntimeError(msg) from None

                # If the process is still running, check if it has exceeded the timeout.
                if self.timeout is None:
                    continue
                running_time = time.time() - start_time  # Determine the running time.
                if running_time > self.timeout:
                    print(f"Execution exceeded timeout of {self.timeout}s")  # Log a timeout message.
                    os.kill(self.process.pid, signal.SIGINT)  # Send SIGINT to the process.
                    child_in_overtime = True  # Mark that the process is now in overtime.

                    # If the process exceeds the timeout by more than 5 seconds, force cleanup.
                    if running_time > self.timeout + 5:
                        self.cleanup_session()  # Clean up the child process.

                        state = (None, "TimeoutError", {}, [])  # Set state to indicate a timeout error.
                        exec_time = self.timeout  # Set the execution time to the timeout limit.
                        break

        output: list[str] = []  # Initialize a list to collect output lines.
        # Collect all output from the result queue until the EOF marker is encountered.
        start_collect = time.time()  # Record the start time for output collection.
        while not self.result_outq.empty() or not output or output[-1] != "<|EOF|>":
            try:
                # If output collection exceeds 5 seconds, log a warning.
                if time.time() - start_collect > 5:
                    print.warning("Output collection timed out")
                    break
                output.append(self.result_outq.get(timeout=1))  # Append the next line of output.
            except queue.Empty:
                continue  # Continue if no output is available immediately.
        output.pop()  # Remove the EOF marker from the output list.

        # Extract exception information from the finished state.
        e_cls_name, exc_info, exc_stack = state[1:]

        if e_cls_name == "TimeoutError":
            # Append a timeout error message to the output if a timeout occurred.
            output.append(
                f"TimeoutError: Execution exceeded the time limit of {humanize.naturaldelta(self.timeout)}"
            )
        else:
            # Append the execution time information to the output.
            output.append(
                f"Execution time: {humanize.naturaldelta(exec_time)} seconds (time limit is {humanize.naturaldelta(self.timeout)})."
            )
        # Return an ExecutionResult object with all the execution details.
        return ExecutionResult(output, exec_time, e_cls_name, exc_info, exc_stack)



### Nodes


In [ ]:
import time
import uuid
from dataclasses import dataclass, field
#注意，这里引入了一个名为 field 的数据类
from typing import Literal, Optional
from dataclasses_json import DataClassJsonMixin


@dataclass(eq=False) #禁用了数据类型之间相等的比较关系
class Node(DataClassJsonMixin):
    """A single node in the solution tree. Contains code, execution results, and evaluation information."""
    # field 方法提供了更加精细化的属性控制
    # 例如下面的属性中基本都有 kw_only=True 这一词缀，这说明在初始化该属性时必须具备关键词(key word)，而不能使用位置对应的方式来访问该属性
    # 被 field 修饰的这种特殊的属性，也叫做“字段”，下面将这些具备field特性的属性称为字段。
    # ---- code & plan ----
    code: str
    plan: str = field(default=None, kw_only=True)  # type: ignore
    #plan的默认值是 None，调用该属性时必须声明其名字 plan,一个例子如下
    """
    正确用法
    node = Node(code="print('hello')", plan="My plan")
    错误用法
    node = Node("print('hello')", "My plan")
                                   ^^^这会报错
    """
    # ---- general attrs ----
    step: int = field(default=None, kw_only=True)  # type: ignore
    id: str = field(default_factory=lambda: uuid.uuid4().hex, kw_only=True)
    #这里展示了field的另外一个能力：为字段赋予一个方法。
    #uuid.uuid4().hex是一个生成随机数的方法
    #同时还要注意这里用的是default_factory，而不是default，这是为了每个实例创建的时候都有一个不同的值。
    #如果是default，那么每个实例的id字段全是相同的值
    ctime: float = field(default_factory=lambda: time.time(), kw_only=True)
    parent: Optional["Node"] = field(default=None, kw_only=True)
    #Optional意味着这字段可以是Node对象，也可以是None
    children: set["Node"] = field(default_factory=set, kw_only=True)
    #set的特性：无序与不可重复
    """
    上面这个关于父节点和子节点配置的设置是非常实用的设计思想。
    除去Optional和set设置对父节点和子节点特性的高度抽象概括，这里还引入了forward reference，也就是我们看到的使用字符串 “Node” 预先引用了Node类。
    这样可以避免在Node还没生成完毕时引用Node而产生的错误。
    这套设计生成了树状结构，在各种算法设计中，这是非常实用的操作。
    """
    # ---- execution info ----
    _term_out: list[str] = field(default=None, kw_only=True)  # type: ignore
    exec_time: float = field(default=None, kw_only=True)  # type: ignore
    exc_type: str | None = field(default=None, kw_only=True)
    # str | None 等价于 Optional[str]，当执行没有错误的时候，这个字段就是None。同时要注意这个特性是Py3.10+的，所以该项目的py版本>=10
    exc_info: dict | None = field(default=None, kw_only=True)
    exc_stack: list[tuple] | None = field(default=None, kw_only=True)

    # ---- evaluation ----
    # post-execution result analysis (findings/feedback)
    analysis: str = field(default=None, kw_only=True)  # type: ignore
    metric: float = field(default=None, kw_only=True)  # type: ignore
    # whether the agent decided that the code is buggy
    # -> always True if exc_type is not None or no valid metric
    is_buggy: bool = field(default=None, kw_only=True)  # type: ignore
    improve_flag: bool = field(default=None, kw_only=False)

    def __post_init__(self) -> None:
        if self.parent is not None:
            self.parent.children.add(self)

    @property
    def stage_name(self) -> Literal["draft", "debug", "improve"]:
        """
        Return the stage of the node:
        - "stage" if the node is an initial solution draft
        - "debug" if the node is the result of a debugging step
        - "improve" if the node is the result of an improvement step
        """
        if self.parent is None:
            return "draft"
        #不存在父节点时，该node处于draft stage(草稿阶段）
        return "debug" if self.parent.is_buggy else "improve"
        #父节点出错时，该节点进入 debug stage，否则进入improve stage
        # property装饰器可以让我们直接通过像属性一样访问方法
        #具体来说，就是 node.stage_name 就可以直接访问到这个方法了，而不需要 node.stage_name()


    def absorb_exec_result(self, exec_result: ExecutionResult):
        """Absorb the result of executing the code from this node."""
        self._term_out = exec_result.term_out
        self.exec_time = exec_result.exec_time
        self.exc_type = exec_result.exc_type
        self.exc_info = exec_result.exc_info
        self.exc_stack = exec_result.exc_stack

    @property
    def term_out(self) -> str:
        """Get the terminal output of the code execution (after truncating it)."""
        return trim_long_string("".join(self._term_out))

    @property
    def is_leaf(self) -> bool:
        """Check if the node is a leaf node in the solution tree."""
        return not self.children

    def __eq__(self, other):
        return isinstance(other, Node) and self.id == other.id

    def __hash__(self):
        return hash(self.id)

    @property
    def debug_depth(self) -> int:
        """
        Length of the current debug path
        - 0 if the node is not a debug node (parent is not buggy)
        - 1 if the parent is buggy but the skip parent isn't
        - n if there were n consecutive debugging steps
        """
        if self.stage_name != "debug":
            return 0
        return self.parent.debug_depth + 1  # type: ignore

### Tree

In [ ]:
@dataclass
class Journal(DataClassJsonMixin):
    """节点串起来，形成了“树”"""

    nodes: list[Node] = field(default_factory=list)
    #变量名: 类型 = 值，类似函数的注解，这是对变量的注解。
    #这里再次调用了default_factory也是一样的原因，避免所有journal的nodes都相同。

    def __getitem__(self, idx: int) -> Node:
        return self.nodes[idx]

    def __len__(self) -> int:
        """Return the number of nodes in the journal."""
        return len(self.nodes)

    def append(self, node: Node) -> None:
        """Append a new node to the journal."""
        node.step = len(self.nodes)
        self.nodes.append(node)

    @property
    def draft_nodes(self) -> list[Node]:
        """Return a list of nodes representing intial coding drafts"""
        return [n for n in self.nodes if n.parent is None]
        #经典的列表推导式
        """
        第一个n代表即将放入这个列表的内容，为什么我知道是列表，是因为最外侧的两个[]
        第二个n就是循环变量，当然这个循环变量最后是要放入列表的
        在哪里循环？在node.parent=None的地方循环
        也就是把所有没有父节点的Node视为draft.
        """

    @property
    def buggy_nodes(self) -> list[Node]:
        """Return a list of nodes that are considered buggy by the agent."""
        return [n for n in self.nodes if n.is_buggy]

    @property
    def good_nodes(self) -> list[Node]:
        """Return a list of nodes that are not considered buggy by the agent."""
        return [n for n in self.nodes if not n.is_buggy]

    def get_metric_history(self) -> list[float]:
        """Return a list of all metric values in the journal."""
        return [n.metric for n in self.nodes]

    def get_good_nodes(self) -> Node:
        return [n for n in self.nodes if not n.is_buggy]
        #good nodes 的定义就是没有出BUG的节点

    def get_best_node(self, only_good=True) -> None | Node:
        """Return the best solution found so far (node with the highest validation metric)."""
        if only_good:
            nodes = self.good_nodes
            if not nodes:
                return None
        else:
            nodes = self.nodes
        return min(nodes, key=lambda n: n.metric)

    def generate_summary(self, include_code: bool = False) -> str:
        """Generate a summary of the journal for the agent."""
        summary = []
        for n in self.good_nodes:
            summary_part = f"Design: {n.plan}\n"
            if include_code:
                summary_part += f"Code: {n.code}\n"
            summary_part += f"Results: {n.analysis}\n"
            summary_part += f"Validation Metric (Mean Squared Error): {n.metric}\n"
            summary.append(summary_part)
        return "\n-------------------------------\n".join(summary)

### Agent

In [ ]:
!pip install pydantic

In [ ]:
import random
from typing import Any, Callable, cast

import re
import sys
import json
import humanize

from pydantic import BaseModel


ExecCallbackType = Callable[[str, bool], ExecutionResult]


class Agent:
    def __init__(
        self,
        cfg,
        journal: Journal,
        #Agent类的参数列表
    ):
        super().__init__()
        self.cfg = cfg
        self.journal = journal
        self.data_preview: str | None = None
        #给参数列表赋值

    def search_policy(self) -> Node | None:
        """Select a node to work on (or None to draft a new node)."""
        search_cfg = self.cfg.agent.search

        # initial drafting
        if len(self.journal.draft_nodes) < search_cfg.num_drafts:
            return None
            #如果 draft_nodes(记得吗？之前这里是一个列表推导式）的长度，即draft node的个数小于设定值
            #那么返回None
            #在step函数中，search_policy = None 会把当前Node设置为 draft node.

        # debugging
        if random.random() < search_cfg.debug_prob:
            # nodes that are buggy + leaf nodes + debug depth < max debug depth
            debuggable_nodes = [
                n
                for n in self.journal.buggy_nodes
                if n.is_leaf
            ]
            #只从叶节点中挑buggy节点
            #一般而言LLM生成的第一版代码都没啥问题，可能只是在考虑改进的时候会发生一些问题。
            if debuggable_nodes:
                return random.choice(debuggable_nodes)
            #非常有趣的一点是，这里居然是依靠概率来决定是否进行调试的，而不是根据程序是否出现BUG
            #我个人的猜想：不是所有的BUG都值得修复，也许新创建一个算法更值得尝试
            #如果程序死磕在一个BUG上，资源都被浪费了
            #这种设计或许是能够提高程序产出高品质代码的一种考虑


        # back to drafting if no nodes to improve
        good_nodes = self.journal.good_nodes
        if not good_nodes:
            return None

        # greedy
        greedy_node = self.journal.get_best_node()

        return greedy_node


    def plan_and_code_query(self, system_message, user_message, retries=3) -> tuple[str, str]:
        """
        Generate a natural language plan + code in the same LLM call and split them apart.
        注：这是一个统一的输入接口，不同的agent最终都会把信息输入到这个接口上来，这个接口再与本地的LLM进行交互，输出结果
        """
        completion_text = None
        for _ in range(retries):
            response = generate_response(
                myModel,
                _messages=[
                    {'role': 'system', "content": system_message},
                    {'role': 'user', "content": user_message}
                ]
            )
            completion_text = response
            code = extract_code(completion_text)
            nl_text = extract_text_up_to_code(completion_text)
            #这里还对输出进行了一点处理，也就是将其分成了代码块部分和自然语言部分，实现在下面的 text processing 部分
            if code:
                return nl_text, code
            print("Plan + code extraction failed, retrying...")
        print("Final plan + code extraction attempt failed, giving up...")
        return "", completion_text

    def _draft(self) -> Node:
        # ================ TODO: ask LLM agents to come up with a solution and then implement ================
        system_prompt = "你是一个提出机器学习相关解决方案的机器人"
        user_prompt = [ #可以尝试着注释掉这个部分
            "你必须针对下列任务提出一个优秀的机器学习解决方案，该机器学习必须以python的形式实现"
            f"任务是{str(self.cfg.task_goal)} ",
            f'所有的数据在路径"{self.cfg.data_dir}"下',
            f"{str(self.data_preview)}",
            '你必须把对test集的预测结果放在目录"/content/submissionRes.csv"下.',
            "Note that the testing file DOES NOT have the target column."
        ]
        system_message = system_prompt
        user_message = "\n".join(user_prompt)
        plan, code = self.plan_and_code_query(system_message=system_message, user_message=user_message)
        return Node(plan=plan, code=code)

    def _improve(self, parent_node: Node) -> Node:

        # ================  TODO: ask LLM agent to improve drafts ================

        system_prompt = "你是一个专门优化代码性能的的人工智能"

        user_prompt = [
            f"任务描述: {str(self.cfg.task_goal)} "
            f"这是已有的记忆: {str(self.journal.generate_summary())} "
            f"待优化的代码 {str(wrap_code(parent_node.code))} "
        ]
        system_message = system_prompt
        user_message = " ".join(user_prompt)
        plan, code = self.plan_and_code_query(system_message=system_message, user_message=user_message)
        return Node(plan=plan, code=code, parent=parent_node, improve_flag=True)

    def _debug(self, parent_node: Node) -> Node:

        # ================  TODO: ask LLM agent to debug ================
        system_prompt = "你是一个用于解决代码BUG的人工智能"


        user_prompt = [
            f"任务描述: {str(self.cfg.task_goal)}\n\n",
            f"之前的错误代码是: {str(wrap_code(parent_node.code))}\n\n",
            f"代码的处理结果是: {str(wrap_code(parent_node.term_out, lang=''))}\n\n",
            str(self.data_preview)
        ]

        system_message = system_prompt
        user_message = " ".join(user_prompt)

        plan, code = self.plan_and_code_query(system_message=system_message, user_message=user_message)
        return Node(plan=plan, code=code, parent=parent_node)

    def update_data_preview(
        self,
    ):
        self.data_preview = data_preview_generate(cfg.data_dir)

    def step(self, exec_callback: ExecCallbackType):
    # 执行一个步骤，选择合适的节点进行 draft、debug 或 improve 操作。
    # 如果没有节点或数据预览为空，则生成数据预览
        if not self.journal.nodes or self.data_preview is None:
            self.update_data_preview()

        # 根据搜索策略选择父节点
        parent_node = self.search_policy()

        # 如果没有父节点，则创建一个草稿节点
        if parent_node is None:
            result_node = self._draft()

        # 如果父节点有 bug，则进行 debug 操作
        elif parent_node.is_buggy:
            result_node = self._debug(parent_node)

        # 如果父节点没有 bug 且尚未被改进，则进行 improve 操作
        elif not parent_node.improve_flag:
            result_node = self._improve(parent_node)

        # 如果父节点已经被改进过，则跳过
        else:
            print(f"Node {parent_node} 已经被改进过，跳过此节点。")
            return

        # 解析执行结果并将结果节点添加到日志中
        self.parse_exec_result(
            node=result_node,
            exec_result=exec_callback(result_node.code, True),
        )
        self.journal.append(result_node)


    # def step(self, exec_callback: ExecCallbackType):
    #     #Agent.step 是决定程序运行步数的关键参数，它决定了调用几次agent
    #     if not self.journal.nodes or self.data_preview is None:
    #         #case:1 如果不存在节点或者不存在 data_preview, 那么先生成一个数据preview
    #         self.update_data_preview()

    #     parent_node = self.search_policy()
    #     #现在有节点了，开始进行搜索策略
    #     if parent_node is None:
    #         result_node = self._draft()
    #         #节点没有父节点，那么节点就是草稿节点
    #     elif parent_node.is_buggy:
    #         result_node = self._debug(parent_node)
    #         #节点有bug，那么节点需要进行debug操作
    #     else:
    #         result_node = self._improve(parent_node)
    #         #如果节点有父节点，并且还有没BUG，那么进行improve操作
    #     self.parse_exec_result(
    #         #将节点类别分类完后，送到代码运行结果评估agent
    #         node=result_node,
    #         exec_result=exec_callback(result_node.code, True),
    #     )
    #     self.journal.append(result_node)


    def parse_exec_result(self, node: Node, exec_result: ExecutionResult):
        node.absorb_exec_result(exec_result)
        system_prompt = """
                你是一个评估机器学习成果的人工智能。
                请分析代码执行结果，并以JSON格式返回以下信息：
                1. summary: 对代码执行结果的简要分析
                2. is_buggy: 代码是否存在bug (true/false)
                3. metric: 如果有性能指标，请提取数值(如MSE等)；如果没有有效的MSE，则设置MSE=3.0

                你的回答必须是一个有效的JSON对象，格式如下:
                {
                    "summary": "...",
                    "is_buggy": true/false,
                    "metric": 正确数值或3.0
                }
                """
        # ================  TODO: ask LLM agent to extract evaluation result from the execution output. ================
        # save log file
        user_prompt = f"""
            任务描述是:
            {self.cfg.task_goal}
            代码实现是:
            {wrap_code(node.code)}
            代码的输出结果是:
            {wrap_code(node.term_out, lang="")}
        """
        print("传入parse_exer_result的执行结果是:",node.term_out)
        system_message = system_prompt
        user_message = " ".join(user_prompt)

        response_text = generate_response(
            myModel,
            _messages=[
                {'role': 'system', "content": system_message},
                {'role': 'user', "content": user_message}
            ]
        )
    # 解析 JSON 数据
        try:
            response_text = response_text.strip("```json").strip("```").strip()
            # 提取 JSON 对象
            response_json = json.loads(response_text)

            # 验证 JSON 格式是否包含所需字段
            if not all(key in response_json for key in ["summary", "is_buggy", "metric"]):
                raise ValueError("返回的 JSON 缺少必要字段")

            # 更新 Node 的字段
            node.analysis = response_json["summary"]
            node.is_buggy = response_json["is_buggy"]
            node.metric = response_json["metric"]
            # 将 JSON 数据保存到文件
            with open("execution_result.json", "w", encoding="utf-8") as json_file:
                json.dump(response_json, json_file, ensure_ascii=False, indent=4)

        except json.JSONDecodeError:
            # 如果解析失败，记录错误并标记为有问题的节点
            print(f"JSON 解析失败: {response_text}")
            node.analysis = "LLM 返回的结果无法解析为 JSON"
            node.is_buggy = True
            node.metric = 3.0
        except ValueError as e:
            # 如果 JSON 格式不符合要求，记录错误并标记为有问题的节点
            print(f"JSON 格式错误: {e}")
            node.analysis = "LLM 返回的 JSON 不符合预期格式"
            node.is_buggy = True
            node.metric = 3.0
        except Exception as e:
            # 捕获其他异常
            print(f"未知错误: {e}")
            node.analysis = "评估过程中发生未知错误"
            node.is_buggy = True
            node.metric = 3.0


### Text Processing

In [ ]:
import json
import re

def wrap_code(code: str, lang="python") -> str:
    """Wraps code with three backticks."""
    return f"```{lang}\n{code}\n```"


def is_valid_python_script(script):
    """Check if a script is a valid Python script."""
    try:
        compile(script, "<string>", "exec")
        return True
    except SyntaxError:
        return False


def extract_jsons(text):
    """Extract all JSON objects from the text. Caveat: This function cannot handle nested JSON objects."""
    json_objects = []

    # Find {} by regular expression
    matches = re.findall(r"\{.*?\}", text, re.DOTALL)

    # Try to transform string into json objects
    for match in matches:
        try:
            json_obj = json.loads(match)
            json_objects.append(json_obj)
        except json.JSONDecodeError:
            pass

    return json_objects

def trim_long_string(string, threshold=5100, k=2500):
    # Check if the length of the string is longer than the threshold
    if len(string) > threshold:
        # Output the first k and last k characters
        first_k_chars = string[:k]
        last_k_chars = string[-k:]

        truncated_len = len(string) - 2 * k

        return f"{first_k_chars}\n ... [{truncated_len} characters truncated] ... \n{last_k_chars}"
    else:
        return string

def extract_code(text):
    """Extract python code blocks from the text."""
    parsed_codes = []

    # When code is in a text or python block
    matches = re.findall(r"```(python)?\n*(.*?)\n*```", text, re.DOTALL)
    for match in matches:
        code_block = match[1]
        parsed_codes.append(code_block)

    # When the entire text is code or backticks of the code block is missing
    if len(parsed_codes) == 0:
        matches = re.findall(r"^(```(python)?)?\n?(.*?)\n?(```)?$", text, re.DOTALL)
        if matches:
            code_block = matches[0][2]
            parsed_codes.append(code_block)

    # validate the parsed codes
    valid_code_blocks = [
        c for c in parsed_codes if is_valid_python_script(c)
    ]
    return "\n\n".join(valid_code_blocks)

def extract_text_up_to_code(s):
    """Extract (presumed) natural language text up to the start of the first code block."""
    if "```" not in s:
        return ""
    return s[: s.find("```")].strip()



### Feature Selection

In [ ]:
!pip install pandas

In [ ]:
import json
from pathlib import Path

import humanize
import pandas as pd


def preview_csv(p: Path) -> str:
    """Generate a textual preview of a csv file"""

    df = pd.read_csv(p)

    out = []

    out.append(f"-> {str(p)} has {df.shape[0]} rows and {df.shape[1]} columns.")

    # ================  TODO: Tell LLM agents which feature is useful for prediction ================

    cols = df.columns.tolist()
    cols_str = ", ".join(cols)
    res = f"The columns are: {cols_str}"

    out.append(res)

    return "\n".join(out)

def data_preview_generate(base_path):
    """
    Generate a textual preview of a directory
    """

    result = []
    files = [p for p in Path(base_path).iterdir()]

    if not files:
        return "No CSV files found in the specified directory."

    for f in sorted(files):
        try:
            result.append(preview_csv(f))
        except Exception as e:
            result.append(f"Error processing {f}: {str(e)}")

    result = "\n\n".join(result)

    return result


## Config

In [ ]:
import torch
import random
import numpy as np

# DO NOT MODIFY THIS CELL
class Config:
    """
    A recursive configuration class that converts a dictionary into an object
    with attributes accessible using dot notation.
    """

    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                value = Config(value)
            setattr(self, key, value)

def set_seed(seed=531):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed()

In [ ]:
# ================  TODO: config ================
config = {
    # experiment configurations
    "exp_name": "ML2025_HW2",
    "data_dir":  Path("/content/ML2025Spring-hw2-public").resolve(),

    # the description of the task
   "task_goal":
    """
  你的任务是基于前两天的调查结果和行为数据，预测美国各州在第3天检测出COVID-19阳性的概率。
  这是一个回归的机器学习任务，请不要使用分类相关的机器学习算法。
  给定的数据集中包含多个特征，但并非所有特征都与预测目标相关或适合建模。你需要对数据集进行分析，识别并选择对 testing_positive_day3 最有影响的特征。

  在完成任务时，请注意以下要求：
  1. 特征选择时，保留所有与地理区域相关的特征（如各州的编码特征，例如 AL、CA 等）。
  2. 尝试多种回归模型（例如SVR、随机森林回归、梯度提升回归等），并通过超参数调优选择性能最佳的模型。
  3. 使用交叉验证和网格搜索进行超参数调优，输出每个模型的最佳参数和对应的验证集 Mean Squared Error (MSE)。
  4. 数据集是完整的，请不要在结果中输出数据集内容。
  5. 最终，请对 Test 集进行预测，并保存预测结果。
    """,

    "agent": {
        # the number of iterations
        "steps": 20,
        "search": {
            # decide whether to debug or improve
            "debug_prob": 0.5,
            # the number of draft generated before improving/debugging
            "num_drafts": 3,
        },
    },
}

cfg = Config(config)

## Main

In [ ]:
from tqdm import tqdm
def main():

    def exec_callback(*args, **kwargs):
        res = interpreter.run(*args, **kwargs)
        return res

    journal = Journal()
    agent = Agent(
        cfg=cfg,
        journal=journal,
    )

    interpreter = Interpreter()

    global_step = len(journal)
    while global_step < cfg.agent.steps:
        print(f"Step {global_step + 1}/{cfg.agent.steps} in progress...")
        # run agent
        agent.step(exec_callback=exec_callback)
        # display the latest node
        latest_node = journal.nodes[-1]  # 获取最新的节点
        print(f"Step {global_step + 1} completed.")
        print(f"Latest Node ID: {latest_node.id}")
        # print(f"Plan: {latest_node.plan}")
        # print(f"Code:\n{wrap_code(latest_node.code)}")
        print(f"Metric: {latest_node.metric}")
        print(f"Is Buggy: {latest_node.is_buggy}")
        print(f"Execution Time: {latest_node.exec_time:.2f} seconds" if latest_node.exec_time else "Execution Time: None")
        if latest_node.parent:
            parent_node = latest_node.parent
            print(f"Parent Node ID: {parent_node.id}")
            # print(f"Parent Plan: {parent_node.plan}")
            print(f"Parent Metric: {parent_node.metric}")
            print(f"Parent Is Buggy: {parent_node.is_buggy}")
        else:
            print("Parent Node: None")
        print("-" * 50)
        # save results for this iteration
        save_run(cfg, journal)
        # get current step
        global_step = len(journal)

    # Kill created child process
    interpreter.cleanup_session()


    # Kill created child process
    interpreter.cleanup_session()


if __name__ == "__main__":
    main()


Step 1/20 in progress...
传入parse_exer_result的执行结果是: Best parameters for SVR: {'C': 1, 'epsilon': 0.2, 'kernel': 'linear'}
Best MSE for SVR: 0.9960581929309686
Best parameters for RandomForest: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Best MSE for RandomForest: 1.072458657290075
Best parameters for GradientBoosting: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
Best MSE for GradientBoosting: 1.0700934080592348
Cross-validated MSE for SVR: 0.9960581929309686
Cross-validated MSE for RandomForest: 1.0716296798377551
Cross-validated MSE for GradientBoosting: 1.0706121784804945
Execution time: 9 minutes seconds (time limit is 20 minutes).
Step 1 completed.
Latest Node ID: 77118aed0b2b4ed6be256a3291ad7c77
Metric: 0.996
Is Buggy: False
Execution Time: 575.42 seconds
Parent Node: None
--------------------------------------------------
Step 2/20 in progress...
传入parse_exer_result的执行结果是: Selected Features: Index(['AL', 'AZ', 'CA', 'CO', 'CT', 'FL', 'GA', 'IL', '

KeyboardInterrupt: 

In [ ]:
!lsof -i :42107

COMMAND      PID       USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
jupyter-n 156599 shijunjian   27u  IPv4 2571145      0t0  TCP localhost:35676->localhost:42107 (ESTABLISHED)
python    165177 shijunjian   10u  IPv4 2578163      0t0  TCP localhost:42107 (LISTEN)
python    165177 shijunjian   17u  IPv4 2579289      0t0  TCP localhost:35668->localhost:42107 (ESTABLISHED)
python    165177 shijunjian   18u  IPv4 2579290      0t0  TCP localhost:42107->localhost:35668 (ESTABLISHED)
python    165177 shijunjian   48u  IPv4 2579293      0t0  TCP localhost:42107->localhost:35676 (ESTABLISHED)


In [ ]:

# Get your best result!
!python best_solution-features.py
# !python DSR1.py

Selected Features: Index(['cli_day1', 'ili_day1', 'wnohh_cmnty_cli_day1',
       'wlarge_event_indoors_day1', 'wshop_indoors_day1',
       'wrestaurant_indoors_day1', 'hh_cmnty_cli_day1', 'nohh_cmnty_cli_day1',
       'tested_positive_day1', 'cli_day2', 'ili_day2', 'wnohh_cmnty_cli_day2',
       'wlarge_event_indoors_day2', 'wshop_indoors_day2',
       'wrestaurant_indoors_day2', 'hh_cmnty_cli_day2', 'nohh_cmnty_cli_day2',
       'tested_positive_day2', 'cli_day3', 'ili_day3', 'wnohh_cmnty_cli_day3',
       'wshop_indoors_day3', 'wrestaurant_indoors_day3', 'hh_cmnty_cli_day3',
       'nohh_cmnty_cli_day3'],
      dtype='object')
Selected Features (sorted by importance): [('tested_positive_day2', np.float64(253793.43948971806)), ('tested_positive_day1', np.float64(104538.47795439813)), ('hh_cmnty_cli_day1', np.float64(19075.52592634041)), ('nohh_cmnty_cli_day1', np.float64(18639.652282689876)), ('hh_cmnty_cli_day2', np.float64(16166.43736104487)), ('wnohh_cmnty_cli_day1', np.float64(160

# References
The code scripts are from [aideml](https://github.com/WecoAI/aideml) project on github with some modifications.

AIDE: AI-Driven Exploration in the Space of Code
https://arxiv.org/pdf/2502.13138
